<a href="https://colab.research.google.com/drive/1KH18f5y9rbvEga6aDdUV907E-rxw453D?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Web Data via Scraping and Automation

### Part 1: Where is my data, and how can I access it?

James Bowden, 06/2020. This tutorial is the first of a two part series on how to scrape web data. This series was written as a part of Caltech's CS42 course (Computer Science Education).

#### Why do we scrape?

With the growth of the internet, there's more data online than we know what to do with! This data can be very useful if we know how to access the bits and pieces that we actually want. 

Of course, we could always just go to a website and extract the data we want ourselves. It's simple enough to download a file or folder and copy/paste a few values. 

For this tutorial, I'll use the NCBI database (https://www.ncbi.nlm.nih.gov/), which contains sequence data for various organisms (among other things), as our example. However, web scraping can be done on almost any website, any STEM (or other) field, or any datatype. Feel free to follow along using a different webpage more relevant to your interests. A few examples of other websites you could extract data from:

*   Astrophysics data (https://ssd.jpl.nasa.gov/?sat_phys_par): might want to compare the radii of various planetary satellites against their gravitational pull.
*   Census data (https://www.census.gov/quickfacts/CA): might want to compare housing, employment, population etc. for different states. Can just switch out state abbrieviations in the URL, making it easy to get the same pieces of information for all 50 states. 
*   Stock market data (https://finance.yahoo.com/quote/AMZN/history?p=AMZN): might want to get stock price data at regular intervals, or get historical data for many stocks all at once for comparison. Scraping makes this convenient since for different stock tickers, you can use the same URL and change out the ticker (e.g. 'AMZN' -> 'GOOG').

Consider the following screenshots, from the NCBI database. The top (https://www.ncbi.nlm.nih.gov/assembly/GCF_000005845.2) is a webpage for a type of *E. coli* (a common bacterium), and the bottom (https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/) is a collection of files that can be downloaded for various parts of the DNA sequence, that we get by clicking "FTP directory for RefSeq assembly" on the top page. For the less biologically inclined, the top is the webpage for an item of interest, and the bottom is a linked online directory with some related data files.

![](https://drive.google.com/uc?export=view&id=13HQhuMG-ChdtQXCUHpWkLlU2NrxG13mt)
![](https://drive.google.com/uc?export=view&id=1M5-jXSlqNi66kdv1M0WBeu1kKd9gVYiq)

Now if I told you to obtain the total sequence length and download the sequence for the coding genes (file that ends in "_cds_from_genomic.fna.gz"), you could do it in less than a minute manually. Just copy the value next to "Total sequence length", follow the link, and click on the file you want to download.

##### The problem at hand

Let me pose a slightly more complex problem to you: Let's say I want to go through every type of *E. coli* on the website and make a distribution of the sequence lengths for the whole sequence and some genes (substring of sequence). This is a really simple task behind the biology: given a bunch of strings of characters, find the relative standard deviation. However, **that data isn't organized nicely for me**, which means I need to gather a string from each webpage shown below (from its own file too!). This is nontrivial! *How do we extract the data?*

Feel free to skip this part if it doesn't make sense, but for those who would like a little more reasoning for the example task:

Once we've calculated the relative standard deviation for the total sequence lengths (given as a number in the table above) across all types of *E. coli* listed on the page below, we can compare it to that of individual genes (substring of total sequence). This could give me an idea of whether a particular gene is subject to variations in length more or less than the genome as a whole, which might help me figure out which genes are likely more important to conserve (would have lower relative variation since significant change would kill the organism) vs. more likely disposable (larger variation allowed since organism doesn't depend on gene being correct/there at all). While this is a metric I just came up with and not legitimate in any sense, determinining which genes are essential for life and which are not is currently an unsolved question and is actually being researched in the synthetic biology lab I work in. Most of the metrics used to identify "essential" genes are predictive and unconfirmed and based off of a variety of different data types/sources, so this could be a useful measure. And of course, we will want to be able to do this for a lot of different genes in order to check them. 

**Note that the strength of this analysis is in being able to calculate statistics over a large number of different strains (i.e. similar data from many webpages).** We won't actually complete this analysis in this tutorial series, but will learn how to extract the necessary data, after which this becomes a simple coding problem.

![](https://drive.google.com/uc?export=view&id=115Q8GTY9j1MWkalBYveQ9t1MvgVVE5pm)


Above is the search page in the NCBI assembly database for "e. coli" (https://www.ncbi.nlm.nih.gov/assembly/?term=e.+coli). The strain we checked out earlier is the second entry here...of **19988** total assemblies. There's no way we can click that many times, but there's a lot of useful information we can get from comparing many strains. 

**Having to obtain data manually takes a lot of time, and limits the amount of data that we can actually use!!** Would you even dream of using data from more than a few webpages if you knew you had to follow each link, copy the value, click another link, and download the file? No way! 

##### So what actually is scraping?

**Scraping** is a technique that allows us to get around this obstacle by accessing data from the website via the HTML code that provides the basis for the graphical interface you see. In other words, scraping is used to pull data (both text and files) off of websites *en masse* onto your computer so that you can use it however you'd like! This is incredibly useful because there is so much data online, and it would be a lot of work to obtain (and make use of) it all manually. By scraping, we can get our computer to do the hard (and quite repetitive) work for us so that we can extract and process huge amounts of web data very quickly!

##### Considerations and precautions

I want to make a few quick points before we start. You may notice if you visit the search page I linked above that the NCBI offers a new 'experimental' tool for bulk downloading genomic data (top right of previous image). This is great, but such tools aren't always reliable and often don't let you pick and choose what data you actually want. For example, this particular tool isn't actually working as I write this, and if we just wanted to go through each link and obtain the total sequence length, this method would not be efficient at all (only gives files). For websites where the data that we want is not stored in files but as text/tables, this probably won't even be an option, so scraping is often the most practical choice.

This is an interesting segue into my next point--the NCBI is offering tools like this because they'd rather avoid scraping. This is because scraping allows you to access data from their server much faster than usual. If I could query their server 100 times per second for some period of time, I would probably be able to crash it--which the NCBI wants to avoid. Most web servers are set up to handle a frequency akin to how fast someone can click, not how fast your computer can execute lines of code (*much, much* faster than clicking!!). They try to discourage scraping by offering tools as an alternative, and most good websites will limit the frequency with which you can query their server. In other words, if we try to query the website's server too many times in one minute, it will probably kick us off (or crash, which has roughly the same implications for us).

And lastly, read the website's Terms and Conditions to make sure you can legally use the data how you intend to use it. Many websites won't let you use their data for commercial purposes. I've included the NCBI's policies below; notice that they provide guidelines for how often you can query their server. We'll only need to worry about the first, third, and last bullets.

![](https://drive.google.com/uc?export=view&id=1ssH6r1HhjGTLFaUbRd6WSstb5Zns04g3)

#### Some prerequisites

To follow along with this tutorial on your own computer, you'll need to have a few things set up. First, you'll need a working Python environment. If you don't already have one set up, I recommend downloading and installing Miniconda, which has only the base necessities and allows us to install packages as we need them. You can find the latest version for your OS here: https://docs.conda.io/en/latest/miniconda.html.

As the aim is to scrape web data, we'll also need a web browser. This tutorial will use Chrome, but most standard browsers should have a very similar version of the tools we'll be using. 

Lastly, we'll need to install a few packages. While I'll include the conda command for each package, note that you can easily look up the install commands for whatever package manager you use (e.g. pip, choco, etc.).

#### Where is the data stored?

Let's get into it!

So why is it difficult to extract data from a website? Well it's actually not that hard--it just seems that way because we don't really know how the data is stored and formatted. A webpage is essentially a graphical user interface that displays a bunch of information. This information has to be stored somewhere, but where?

The data for websites is stored on what is called a *server*. This is basically a remote computer that houses data and communicates with the web. When I click on the link for a website, it sends a request to the server and the server sends back the information that I've requested to my browser so that it can render it as a webpage. We can't really do much with the graphical interface that is the webpage itself, but we can extract information from the data that forms the basis for the graphical interface.

The universal convention for storing website data is HTML (Hypertext Markup Language)--websites are "coded" in HTML, and then can be styled via various methods. We can ignore the styling though since we just want the information.

##### The inspect tool

Pretty much every standard browser has a very useful feature called the inspect tool. You can generally access it by right clicking on the website you wish to "inspect" or using the keyboard shortcut. This will generate a side panel in your browser (this is Chrome) which contains the HTML code that corresponds to our graphical website!

![](https://drive.google.com/uc?export=view&id=1yhN6SByZmJ3ZaVcnahq6BZVd7JXPNj7U)

Okay...but there's a lot of information there. How do we know what code corresponds to the information that we want?

If you look closely at the top left corner of the panel, you'll notice that there's a little box with a pointer in it:

![](https://drive.google.com/uc?export=view&id=1X_A_VogirjrOcR--LNjZxTMXeX4tgQ8y)

This is a really handy tool! When we click on this box, we can then select any element on the page and the corresponding HTML lines will be highlighted in the side panel.

We can do this for the total sequence length. We see that the length is stored in between some $<td>$ tags, which denotes that it is a data entry in a table. Note that right above it is the label, "Total sequence length", also between some $<td>$ tags. It's important to take note of such **unique identifiers**, as these will be how we separate out the data we want from the rest of the HTML code.

![](https://drive.google.com/uc?export=view&id=1xbCFsJgWcy55flduEyFxdZRYnmqRN_QH)

We can also do this for the hyperlink to the FTP directory. The url information for this link is stored between some $<a>$ tags. The unique identifier in this case might be the link display name, 'FTP directory for RefSeq assembly' that follows the hyperlink.

![](https://drive.google.com/uc?export=view&id=1fS5rGiOi1lusmh-it936eadYEjwO6uLW)

And again for the file we'd like to download within the FTP directory, which is also contained in a similar format since it's still a hyperlink of sorts (but one that leads to a downloadable file).

![](https://drive.google.com/uc?export=view&id=1OvWlzrdSNeFoUj0vuEpUUnJ2xs3jmCec)

So we know that all of the information that we'd like to obtain is here in the HTML code--the total length is there, the hyperlink to the FTP directory is there, and we can "follow" that link and the hyperlink to the downloadable file we want is also in the HTML for that respective page. This is for an individual webpage, but can be applied to getting all of the individual webpages from the main search page--just inspect the search page and find a unique identifier for the hyperlinks to each individual strain webpage. This is a bit more challenging, but I encourage you to try it out! You'll need to think of something that can be found near every link that you want, but is not found on any of the others (the webpage has lots of other miscellaneous links that we don't care for). 

#### The requests package

We've established that all of the data for the website is stored in some HTML code. Great, but how do we get the HTML code onto our computer so that we can extract the information? For this, we'll use a handy Python package called requests. If you don't already have this package installed, you should do that now.

In [ ]:
# installation command for miniconda/anaconda environment
conda install -c anaconda requests

Once the package is installed, we'll import it and assign the URL for the website we want to scrape from to a variable. Then we can call requests.get(url), which takes in a URL in string form and pulls the HTML data from the website's server. This data is given to us as a response, and if the call was successful in retrieving the data, it should output the following.

In [1]:
import requests

url = 'https://www.ncbi.nlm.nih.gov/assembly/GCF_000005845.2'
response = requests.get(url)
# check that it worked; will only print in jupyter
response

<Response [200]>

Now we have some data from the website stored in our variable response. The response has a few different fields, including the text and encoding. We want the text, as this is the actual HTML data. We can obtain it as follows:

In [ ]:
html_data = response.text
# print out data, truncated because it's pretty long
response.text[:1000] + ' ...'

'<?xml version="1.0" encoding="utf-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">\n    <head xmlns:xi="http://www.w3.org/2001/XInclude"><meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <!-- meta -->\n    <meta name="robots" content="index,nofollow,noarchive" />\n<meta name="ncbi_app" content="entrez" /><meta name="ncbi_db" content="assembly" /><meta name="ncbi_report" content="full" /><meta name="ncbi_format" content="html" /><meta name="ncbi_pagesize" content="20" /><meta name="ncbi_sortorder" content="sortorder" /><meta name="ncbi_pageno" content="1" /><meta name="ncbi_resultcount" content="1" /><meta name="ncbi_op" content="retrieve" /><meta name="ncbi_pdid" content="full" /><meta name="ncbi_sessionid" content="CE8A04A6EF6780E1_1210SID" /><meta name="ncbi_uidlist" content="1755381" /><meta name="ncbi_filter" c

This is the HTML data that encodes for all of the information that we see on the website! You can run this without slicing the result to see all of the webpage's information as one big string. As you've probably noticed, it looks like a bunch of gibberish and isn't incredibly useful in this form. 

##### Looking forward

In the next part of this series (https://colab.research.google.com/drive/1p2qotYuC1VE6sffZWRKoQqdlrgbgLk8v?usp=sharing), we'll learn how to parse and extract desired information from HTML and other data formats so that we can make use of it! We'll also learn how to download files with Python and write some functions to automate this whole process for us. I seriously encourage you to try finding a webpage you'd like to extract data from (or using one of the samples provided) and following along with the tutorial yourself before moving on if you've never scraped before!

In [ ]:
%load_ext watermark
%watermark -v -p requests

CPython 3.7.7
IPython 7.15.0

requests 2.23.0
